In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.metrics import classification_report

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Josvaldes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Josvaldes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# Leer el archivo CSV
df = pd.read_csv('C:/Users/Josvaldes/Documents/Maestria/Austral/2ano/textMining/proyecto/TextMining/TextMining/RECLAMOS 4.csv', encoding='utf-8')
df.head()

,problema_id,descripcion,alias_8_nombre3,problema_n,obsitem,Concaobsitem
0,70e03aaf-6cfb-42e8-92ae-fcb46ff479dd,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.),GN - Gas Natural,B2 - Pérdidas de gas sin piezas dañadas,HAY UNA PERILLA DE LAS HORNALLA QUE NO SE PUED...,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.) GN...
1,e5be47a0-c506-4a79-87e9-2a45f06d496c,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.),GN - Gas Natural,G - Perilla no gira / gira mal,HAY UNA PERILLA DE LAS HORNALLA QUE NO SE PUED...,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.) GN...
2,2b2af341-20c5-4607-baa0-01972617647e,TERMO ELECTRICO EL-55,ELE - Termo eléctrico,D - No enciende,DESPUÉS DE QUE UNA PERSONA SE DUCHO NO VOLVIÓ ...,"TERMO ELECTRICO EL-55 ELE - Termo eléctrico "" ..."
3,ab0ed03a-8aab-4317-9aad-fcc06dd41520,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.),GN - Gas Natural,D2 - Hornalla Mal funcionamiento,LAS HORNALLAS TARDAN MUCHÍSIMO TIEMPO EN PREND...,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.) GN...
4,29ce3db5-f678-4f14-b86e-90c8265daad5,COCINA CANDOR S2 (GN) (U.),GN - Gas Natural,C2 - Horno Mal funcionamiento,EL QUEMADOR DEL HORNO NO MANTIENE LA LLAMA ENC...,"COCINA CANDOR S2 (GN) (U.) GN - Gas Natural "" ..."


In [3]:
# Obtener las columnas de diagnósticos y códigos AIS
descripciones = df['Concaobsitem'].tolist()
etiquetas = df['problema_n'].tolist()

In [4]:
for desc in descripciones:
    if not isinstance(desc, str):
        print(f"Elemento {desc} no es una cadena de texto (str).")

In [5]:
# Codificar las etiquetas como etiquetas numéricas
label_encoder = LabelEncoder()
etiquetas_numericas = label_encoder.fit_transform(etiquetas)

In [6]:
# Preprocesamiento de texto
stemmer = SnowballStemmer('spanish')
stop_words = set(stopwords.words('spanish'))

In [7]:
def preprocesar_texto(texto):
    palabras = nltk.word_tokenize(texto.lower())
    palabras = [stemmer.stem(palabra) for palabra in palabras if palabra.isalpha() and palabra not in stop_words]
    return ' '.join(palabras)

In [8]:
descripciones_preprocesadas = [preprocesar_texto(desc) for desc in descripciones]

In [9]:
# División de los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(descripciones_preprocesadas, etiquetas_numericas, test_size=0.2, random_state=42)

In [10]:
# Creación del pipeline para el clasificador basado en texto
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),  # Experimenta con diferentes valores para max_features
    ('clf', SVC(kernel='linear'))  # Experimenta con diferentes kernels
])

In [11]:
# Entrenamiento del modelo
pipeline.fit(X_train, y_train)
scores = cross_val_score(pipeline, X_train, y_train, cv=5) 

c:\Users\Josvaldes\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


In [12]:
print("Accuracy en validación cruzada:", scores.mean())

Accuracy en validación cruzada: 0.7538706174634751


In [13]:
# Evaluación del modelo en el conjunto de prueba
y_pred = pipeline.predict(X_test)
report = classification_report(y_test, y_pred, labels=np.unique(etiquetas_numericas), target_names=label_encoder.inverse_transform(np.unique(etiquetas_numericas)), zero_division=1)
print(report)

                                                                                                   precision    recall  f1-score   support

                                                                    B - Descargas de electricidad       0.29      0.17      0.21        12
                                               B - La llama no enciende al abrir el grifo de agua       0.76      0.95      0.84        39
                                                      B - Pérdidas de gas sin/con piezas quemadas       1.00      0.00      0.00         6
                                                    B - Pérdidas de gas – sin/con piezas quemadas       1.00      0.00      0.00         7
                                                          B2 - Pérdidas de gas sin piezas dañadas       0.75      0.78      0.76       278
                              C - La llama se apaga luego de algunos minutos de estar funcionando       1.00      0.67      0.80        15
                          

In [14]:
# Cargar descripciones de problemas para evaluar
archivo_evaluacion = 'C:/Users/Josvaldes/Documents/Maestria/Austral/2ano/textMining/proyecto/TextMining/TextMining/testesco.txt'
lineas = pd.read_csv(archivo_evaluacion, delimiter='\t', header=None, names=['Descripcion'])
lineas

,Descripcion
0,"TERMO 45 L GN (U.) GAS - Termo a gas "" EL EQUI..."
1,"TERMO ELECTRICO EL-55 ELE - Termo eléctrico "" ..."
2,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.) GN...
3,"TERMO 80 L GN (U.) GAS - Termo a gas "" NO SE M..."
4,"TERMO ELECTRICO EL-90 ELE - Termo eléctrico "" ..."
5,"COCINA CANDOR S2 (GN) (U.) GN - Gas Natural "" ..."
6,COCINA MASTER S2 BL CLASSIC. (GN.) (U.) GN - G...
7,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.) GN...
8,COCINA MASTER S2 NEG.CLASSIC (GN) (U.) GN - Ga...
9,COCINA MASTER S2 NEG.CLASSIC (GN) (U.) GN - Ga...


In [15]:
resultados = []

for index, row in lineas.iterrows():
    problema = row['Descripcion'].strip()
    problema_preprocesado = preprocesar_texto(problema)
    predicciones = pipeline.predict([problema_preprocesado])
    clase_predicha = label_encoder.inverse_transform(predicciones)[0]
    resultados.append({
        'Problema': problema,
        'Etiqueta de Reclamo': clase_predicha
    })
    

resultados_df = pd.DataFrame(resultados)


In [16]:
archivo_resultado = 'C:/Users/Josvaldes/Documents/Maestria/Austral/2ano/textMining/proyecto/TextMining/TextMining/testresultadoCV_SVC4.txt'
resultados_df.to_csv(archivo_resultado, index=False, sep='\t', encoding='utf-8')

print("Resultados guardados en:", archivo_resultado)

Resultados guardados en: C:/Users/Josvaldes/Documents/Maestria/Austral/2ano/textMining/proyecto/TextMining/TextMining/testresultadoCV_SVC4.txt


In [17]:
# Leer el archivo excel
dfPrueba = pd.read_csv(archivo_resultado, encoding='utf-8', delimiter='\t')
dfPrueba

,Problema,Etiqueta de Reclamo
0,"TERMO 45 L GN (U.) GAS - Termo a gas "" EL EQUI...",C - Pérdidas de agua
1,"TERMO ELECTRICO EL-55 ELE - Termo eléctrico "" ...",C - Pérdidas de agua
2,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.) GN...,E - Puerta de horno - No cierra / Se cae
3,"TERMO 80 L GN (U.) GAS - Termo a gas "" NO SE M...",E - Piloto se apaga
4,"TERMO ELECTRICO EL-90 ELE - Termo eléctrico "" ...",C - Pérdidas de agua
5,"COCINA CANDOR S2 (GN) (U.) GN - Gas Natural "" ...",E - Puerta de horno - No cierra / Se cae
6,COCINA MASTER S2 BL CLASSIC. (GN.) (U.) GN - G...,D1 - Hornalla No enciende
7,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.) GN...,D2 - Hornalla Mal funcionamiento
8,COCINA MASTER S2 NEG.CLASSIC (GN) (U.) GN - Ga...,D3 - Hornalla Se Apaga
9,COCINA MASTER S2 NEG.CLASSIC (GN) (U.) GN - Ga...,B2 - Pérdidas de gas sin piezas dañadas
